In [1]:
# ==============================================================================
# CÉLULA 1: INSTALAÇÃO DE DEPENDÊNCIAS E AUTENTICAÇÃO
# ==============================================================================

print("Instalando dependências...")
!pip install earthengine-api --quiet
!pip install geopandas --quiet
!pip install rasterio --quiet
!pip install shap --quiet
!pip install geemap --quiet
print("Dependências instaladas.")

# ------------------------------------------------------------------------------
# Autenticação e Inicialização de Serviços Google
# ------------------------------------------------------------------------------
from google.colab import drive
import ee

# A autenticação com o Earth Engine
try:
    ee.Initialize(project="the-byway-476116-n7")
    print("API do Google Earth Engine já inicializada.")
except Exception as e:
    print("Autenticando na API do Google Earth Engine...")
    ee.Authenticate()
    ee.Initialize(project="the-byway-476116-n7")
    print("API do Google Earth Engine inicializada.")

# Monta o Google Drive para salvar arquivos (datasets, modelos)
print("Montando Google Drive...")
drive.mount('/content/drive')
print("Google Drive montado em /content/drive")

Instalando dependências...
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 22.3/22.3 MB 89.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 21.6 MB/s eta 0:00:00
Dependências instaladas.
Autenticando na API do Google Earth Engine...
API do Google Earth Engine inicializada.
Montando Google Drive...
Mounted at /content/drive
Google Drive montado em /content/drive


In [2]:
# ==============================================================================
# CÉLULA 2: IMPORTAÇÃO DAS BIBLIOTECAS PRINCIPAIS
# ==============================================================================
import numpy as np
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt
import torch
import geemap

print("Bibliotecas principais importadas. Ambiente pronto para uso!")

Bibliotecas principais importadas. Ambiente pronto para uso!


In [ ]:
# ==============================================================================
# PASSO 1: DEFINIÇÃO DOS PARÂMETROS GLOBAIS
# ==============================================================================

# 1. Período de Tempo para Análise(10 anos)
START_DATE = '2014-01-01'
END_DATE = '2023-12-31'

# 2. Área de Interesse(AOI)
biomas = ee.FeatureCollection('projects/mapbiomas-workspace/AUXILIAR/ESTATISTICAS/COLECAO8/VERSAO-1/refined_biome')
cerrado_aoi = biomas.filter(ee.Filter.eq('NAME_PT_BR', 'Cerrado')).geometry()

# 3. Resolução Espacial(30m)
TARGET_RESOLUTION = 30

print("Parâmetros definidos:")
print(f"Período de análise: {START_DATE} a {END_DATE}")
print("Área de Interesse: Bioma Cerrado")

Parâmetros definidos:
Período de análise: 2014-01-01 a 2023-12-31
Área de Interesse: Bioma Cerrado


In [ ]:
# ==============================================================================
# PASSO 2: CARREGAR ASSETS DE FOCOS DE CALOR
# ==============================================================================
print("Iniciando o carregamento dos assets de focos de calor...")

# 1. Definição caminho-base (pasta) onde os assets estão.
BASE_PATH = 'projects/the-byway-476116-n7/assets/'

# 2. Cria uma lista com os nomes de todos os seus assets anuais.
asset_names = [
    'bdqueimadas_2014-01-01_2014-12-31',
    'bdqueimadas_2015-01-01_2015-12-31',
    'bdqueimadas_2016-01-01_2016-12-31',
    'bdqueimadas_2017-01-01_2017-12-31',
    'bdqueimadas_2018-01-01_2018-12-31',
    'bdqueimadas_2019-01-01_2019-12-31',
    'bdqueimadas_2020-01-01_2020-12-31',
    'bdqueimadas_2021-01-01_2021-12-31',
    'bdqueimadas_2022-01-01_2022-12-31',
    'bdqueimadas_2023-01-01_2023-12-31'
]

# 3. Crie uma lista VAZIA para armazenar os objetos GEE
list_of_collections = []

# 4. Iteração pela lista de nomes para carregar cada asset
print(f"Encontrados {len(asset_names)} nomes de assets. Carregando do GEE...")

for name in asset_names:
    # Constrói o caminho completo do asset
    full_asset_path = BASE_PATH + name

    # Carrega o asset do GEE como uma FeatureCollection
    fc = ee.FeatureCollection(full_asset_path)

    # Adiciona a coleção carregada à nossa lista
    list_of_collections.append(fc)

print(f"Carregou {len(list_of_collections)} coleções de features com sucesso.")

# 5. Junção de todas as coleções em uma única e grande FeatureCollection
active_fires_collection = ee.FeatureCollection(list_of_collections).flatten()
print(active_fires_collection.first().getInfo())

print("\nAssets anuais de focos de calor mesclados com sucesso no servidor!")
print("O objeto 'active_fires_collection' está pronto.")

Iniciando o carregamento dos assets de focos de calor...
Encontrados 10 nomes de assets. Carregando do GEE...
Carregou 10 coleções de features com sucesso.
{'type': 'Feature', 'geometry': {'type': 'Point', 'coordinates': [-44.80399949627662, -11.890001217668136]}, 'id': '0_000000000000000001b7', 'properties': {'Bioma': 'Cerrado', 'DataHora': '2014/01/18 16:39:00', 'DiaSemChuv': 3, 'Estado': 'BAHIA', 'FRP': 8.8, 'Latitude': -11.89, 'Longitude': -44.804, 'Municipio': 'ANGICAL', 'Pais': 'Brasil', 'Precipitac': 1.6, 'RiscoFogo': 0.6, 'Satelite': 'AQUA_M-T'}}

Assets anuais de focos de calor mesclados com sucesso no servidor!
O objeto 'active_fires_collection' está pronto.


In [ ]:
# ==============================================================================
# PASSO 3: CARREGAMENTO DAS COLEÇÕES DE FEATURES
# ==============================================================================

# --- 3.1 DADOS METEOROLÓGICOS (ERA5-Land) ---
# Contém dados de temperatura, umidade, vento, etc.
era5_collection = ee.ImageCollection('ECMWF/ERA5_LAND/HOURLY') \
   .filterDate(START_DATE, END_DATE) \

print(f"Encontradas {era5_collection.size().getInfo()} imagens horárias de meteorologia.")

# --- 3.2 DADOS DE VEGETAÇÃO (MODIS & Landsat) ---
# NDVI/EVI (saúde da vegetação) e LST (temperatura da superfície) do MODIS.
ndvi_collection = ee.ImageCollection('MODIS/061/MOD13A2') \
   .filterDate(START_DATE, END_DATE) \
   .select('NDVI')

lst_collection = ee.ImageCollection('MODIS/061/MOD11A1') \
   .filterDate(START_DATE, END_DATE) \
   .select('LST_Day_1km')

print(f"Encontradas {ndvi_collection.size().getInfo()} imagens de NDVI (16 dias).")
print(f"Encontradas {lst_collection.size().getInfo()} imagens de LST (diárias).")

# --- 3.3 DADOS TOPOGRÁFICOS (SRTM) ---
# Elevação é uma imagem única (estática), não uma coleção.
elevation = ee.Image('USGS/SRTMGL1_003').select('elevation')
# Cálculo do declive (slope) diretamente a partir da elevação.
slope = ee.Terrain.slope(elevation)

print("Dados de elevação e declive carregados.")

# --- .4 DADOS ANTROPOGÊNICOS (MapBiomas) ---
# Uso e cobertura do solo.
land_cover = ee.Image('projects/mapbiomas-public/assets/brazil/lulc/collection10/mapbiomas_brazil_collection10_coverage_v2')
print(land_cover.bandNames().getInfo())

print("Dados de uso e cobertura do solo carregados.")

Encontradas 87624 imagens horárias de meteorologia.
Encontradas 230 imagens de NDVI (16 dias).
Encontradas 3630 imagens de LST (diárias).
Dados de elevação e declive carregados.
['classification_1985', 'classification_1986', 'classification_1987', 'classification_1988', 'classification_1989', 'classification_1990', 'classification_1991', 'classification_1992', 'classification_1993', 'classification_1994', 'classification_1995', 'classification_1996', 'classification_1997', 'classification_1998', 'classification_1999', 'classification_2000', 'classification_2001', 'classification_2002', 'classification_2003', 'classification_2004', 'classification_2005', 'classification_2006', 'classification_2007', 'classification_2008', 'classification_2009', 'classification_2010', 'classification_2011', 'classification_2012', 'classification_2013', 'classification_2014', 'classification_2015', 'classification_2016', 'classification_2017', 'classification_2018', 'classification_2019', 'classification_

In [ ]:
# ==============================================================================
# PASSO 4 : FUNÇÃO MESTRA DE ALINHAMENTO
# ==============================================================================
# Checa se as imagens (LST, NDVI, ERA5) existem (não são nulas)
# Antes de tentar executar o .sample() nelas.

print("Definindo a Função Mestra de Alinhamento (align_all_features) [CORRIGIDA v4]...")

# 1. Renomeia as imagens estáticas ANTES da função
elevation_renamed = elevation.rename('elevation_static')
slope_renamed = slope.rename('slope_static')

static_predictors = ee.Image.cat([elevation_renamed, slope_renamed])


def align_all_features(feature):

    # --- 1. PREPARAÇÃO (Datas e Valores Padrão) ---
    datahora_object = feature.get('DataHora')
    no_data = ee.Number(-9999)
    default_values = {
        'elevation_static': no_data, 'slope_static': no_data, 'lulc_class': no_data,
        'lst_daily_kelvin': no_data, 'ndvi_16day_prefire': no_data, 'temp_2m_C': no_data,
        'relative_humidity_percent': no_data, 'wind_u_ms': no_data,
        'wind_v_ms': no_data, 'wind_speed_ms': no_data, 'precipitation_hourly_mm': no_data
    }

    def process_with_date(date_obj):
        # --- 2. PREPARAÇÃO DAS DATAS ---
        date_string = ee.String(date_obj)
        date_only_string = date_string.slice(0, 10)
        year_string = date_string.slice(0, 4)
        formatted_date_string = date_only_string.replace('/', '-', 'g')
        fire_date = ee.Date(formatted_date_string)
        iso_string_step1 = date_string.replace('/', '-', 'g')
        iso_string_final = iso_string_step1.replace(' ', 'T')
        fire_datetime = ee.Date(iso_string_final)

        # --- 3. ALINHAMENTO ESTÁTICO (Sempre existe) ---
        static_sampled = static_predictors.sample(
            region=feature.geometry(), scale=TARGET_RESOLUTION, numPixels=1
        ).first()

        # --- 4. ALINHAMENTO ANUAL LULC (Sempre existe) ---
        lulc_band_name = ee.String('classification_').cat(year_string)
        lulc_image_this_year = land_cover.select(lulc_band_name)
        lulc_sampled = lulc_image_this_year.sample(
            region=feature.geometry(), scale=TARGET_RESOLUTION, numPixels=1
        ).first()

        # --- 5. ALINHAMENTO DIÁRIO LST (PODE SER NULO) ---
        lst_image_this_day = lst_collection.filterDate(fire_date).first()
        # CORREÇÃO V4: Checa se a imagem existe ANTES de amostrar
        lst_sampled = ee.Algorithms.If(
            lst_image_this_day, # Se a imagem existir...
            ee.Image(lst_image_this_day).sample(region=feature.geometry(), scale=1000, numPixels=1).first(), # ...então amostre
            None # ...senão, retorne nulo
        )

        # --- 6. ALINHAMENTO QUINZENAL NDVI (PODE SER NULO) ---
        search_start_ndvi = fire_date.advance(-30, 'day')
        pre_fire_ndvi_image = ndvi_collection \
                                .filterDate(search_start_ndvi, fire_date) \
                                .sort('system:time_start', False).first()
        # CORREÇÃO V4: Checa se a imagem existe ANTES de amostrar
        ndvi_sampled = ee.Algorithms.If(
            pre_fire_ndvi_image, # Se a imagem existir...
            ee.Image(pre_fire_ndvi_image).sample(region=feature.geometry(), scale=1000, numPixels=1).first(), # ...então amostre
            None # ...senão, retorne nulo
        )

        # --- 7. ALINHAMENTO HORÁRIO ERA5 (PODE SER NULO) ---
        search_start_era5 = fire_datetime.advance(-2, 'hour')
        era5_image_this_hour = era5_collection \
                                .filterDate(search_start_era5, fire_datetime) \
                                .sort('system:time_start', False).first()
        bands_to_sample = [
            'temperature_2m', 'dewpoint_temperature_2m', 'u_component_of_wind_10m',
            'v_component_of_wind_10m', 'total_precipitation_hourly'
        ]
        # CORREÇÃO V4: Checa se a imagem existe ANTES de amostrar
        era5_sampled = ee.Algorithms.If(
            era5_image_this_hour, # Se a imagem existir...
            ee.Image(era5_image_this_hour).select(bands_to_sample).sample(region=feature.geometry(), scale=11132, numPixels=1).first(), # ...então amostre
            None # ...senão, retorne nulod
        )

        # --- 8. EXTRAÇÃO E CÁLCULOS (Agora com segurança) ---
        # (Todos os '..._sampled' podem ser nulos, por isso o 'If' é vital aqui)
        elev = ee.Algorithms.If(static_sampled, static_sampled.get('elevation_static'), no_data)
        slope = ee.Algorithms.If(static_sampled, static_sampled.get('slope_static'), no_data)
        lulc = ee.Algorithms.If(lulc_sampled, lulc_sampled.get(lulc_band_name), no_data)
        lst_k = ee.Algorithms.If(lst_sampled, ee.Number(ee.Feature(lst_sampled).get('LST_Day_1km')).multiply(0.02), no_data)
        ndvi = ee.Algorithms.If(ndvi_sampled, ee.Number(ee.Feature(ndvi_sampled).get('NDVI')).multiply(0.0001), no_data)

        # Converte o resultado de 'era5_sampled' (que pode ser nulo) para um ee.Feature
        # Isso permite que .get() funcione dentro do If
        era5_feature = ee.Feature(era5_sampled)

        temp_c = ee.Algorithms.If(era5_sampled, ee.Number(era5_feature.get('temperature_2m')).subtract(273.15), no_data)
        dew_c = ee.Algorithms.If(era5_sampled, ee.Number(era5_feature.get('dewpoint_temperature_2m')).subtract(273.15), no_data)
        wind_u = ee.Algorithms.If(era5_sampled, era5_feature.get('u_component_of_wind_10m'), no_data)
        wind_v = ee.Algorithms.If(era5_sampled, era5_feature.get('v_component_of_wind_10m'), no_data)
        precip_mm = ee.Algorithms.If(era5_sampled, ee.Number(era5_feature.get('total_precipitation_hourly')).multiply(1000), no_data)

        temp_c_num = ee.Number(temp_c)
        dew_c_num = ee.Number(dew_c)
        wind_u_num = ee.Number(wind_u)
        wind_v_num = ee.Number(wind_v)

        Ea = ee.Number(6.1094).multiply(((ee.Number(17.625).multiply(dew_c_num)).divide(dew_c_num.add(243.04))).exp())
        Es = ee.Number(6.1094).multiply(((ee.Number(17.625).multiply(temp_c_num)).divide(temp_c_num.add(243.04))).exp())
        rh = Ea.divide(Es).multiply(100)
        wind_speed = wind_u_num.pow(2).add(wind_v_num.pow(2)).sqrt()

        # --- 9. RETORNA O FEATURE FINAL E COMPLETO ---
        return feature.set({
            'elevation_static': elev, 'slope_static': slope, 'lulc_class': lulc,
            'lst_daily_kelvin': lst_k, 'ndvi_16day_prefire': ndvi, 'temp_2m_C': temp_c_num,
            'relative_humidity_percent': ee.Algorithms.If(rh.gt(0).And(rh.lte(100)), rh, no_data),
            'wind_u_ms': wind_u_num, 'wind_v_ms': wind_v_num, 'wind_speed_ms': wind_speed,
            'precipitation_hourly_mm': precip_mm
        })

    return ee.Algorithms.If(
        datahora_object,
        process_with_date(datahora_object),
        feature.set(default_values)
    )

# ==============================================================================
# APLICA A FUNÇÃO MESTRA
# ==============================================================================
print("Aplicando a Função Mestra a todos os pontos (definindo a 'receita')...")

dataset_final_positivo = active_fires_collection.map(align_all_features, True)

print("Receita final 'dataset_final_positivo' criada com sucesso.")

Definindo a Função Mestra de Alinhamento (align_all_features) [CORRIGIDA v4]...
Aplicando a Função Mestra a todos os pontos (definindo a 'receita')...
Receita final 'dataset_final_positivo' criada com sucesso.


In [ ]:
# ==============================================================================
# PASSO 5: EXPORTAÇÃO 1 (Amostras Positivas - Features Alinhadas)(FEITO)
# ==============================================================================
print("Iniciando o Passo 5: Preparação da Exportação 1...")
print("A variável 'dataset_final_positivo' (a 'receita' mestra) será enviada para exportação.")

# 1. Definir o nome do arquivo e a pasta no seu Google Drive
output_folder = 'ARTIGO_INCENDIOS_GEE'
output_filename = 'positive_samples_aligned_features'

# 2. Selecionar as colunas que queremos EXPORTAR
columns_to_export = [
    'DataHora', 'Latitude', 'Longitude', 'Municipio', 'Estado', 'Bioma', 'Satelite',
    'elevation_static', 'slope_static', 'lulc_class', 'lst_daily_kelvin',
    'ndvi_16day_prefire', 'temp_2m_C', 'relative_humidity_percent',
    'wind_u_ms', 'wind_v_ms', 'wind_speed_ms', 'precipitation_hourly_mm'
]

# 3. Criar a "Tarefa" (Task) de Exportação
print(f"Definindo a tarefa de exportação para: '{output_folder}/{output_filename}.csv'")

task = ee.batch.Export.table.toDrive(
    collection=dataset_final_positivo,        # A "receita" mestra do Passo 4
    description='Export_Positive_Aligned_v2_MasterFunc',
    folder=output_folder,
    fileNamePrefix=output_filename,
    fileFormat='CSV',
    selectors=columns_to_export
)

# 4. INICIAR a tarefa no servidor do GEE
#task.start()

print("="*60)
print(f"TAREFA ENVIADA! O nome da tarefa é: 'Export_Positive_Aligned_v2_MasterFunc'")
print("="*60)
print("\n>>> INSTRUÇÕES CRÍTICAS (LEIA ABAIXO):")
print("1. Vá para o GEE Code Editor: https://code.earthengine.google.com/")
print("2. Na aba 'TASKS' (Tarefas), encontre 'Export_Positive_..._MasterFunc' (LARANJA).")
print("3. Clique em 'RUN' e confirme.")
print("="*60)

In [ ]:
# ==============================================================================
# PASSO 6.1: CARREGAR DADOS-FONTE ESTÁTICOS
# ==============================================================================
print("Carregando dados-fonte estáticos...")

# 1. Topografia (SRTM - Global)
elevation = ee.Image('USGS/SRTMGL1_003').select('elevation')
print("  Topografia (SRTM) carregada.")

# 2. MapBiomas LULC (Coleção 10)
land_cover = ee.Image('projects/mapbiomas-public/assets/brazil/lulc/collection10/mapbiomas_brazil_collection10_coverage_v2')
print("  Cobertura do Solo (MapBiomas) carregada.")

# 3. Estradas(mapBiomas)
federais = ee.FeatureCollection('projects/the-byway-476116-n7/assets/rodovia-federal')
estaduais = ee.FeatureCollection('projects/the-byway-476116-n7/assets/rodovia-estadual')
outras = ee.FeatureCollection('projects/the-byway-476116-n7/assets/outros-trechos')
todas_estradas = federais.merge(estaduais).merge(outras)

print("  Estradas do assets GEE carregadas.")
# ============================================================

print("Dados-fonte prontos.")

Carregando dados-fonte estáticos...
  Topografia (SRTM) carregada.
  Cobertura do Solo (MapBiomas) carregada.
  Estradas do assets GEE carregadas.
Dados-fonte prontos.


In [ ]:
# ==============================================================================
# PASSO 6.2: ENGENHARIA DE FEATURES ESTÁTICAS(aspect)
# ==============================================================================
print("Iniciando engenharia de features estáticas...")

# --- 1. Lógica do Aspect (Topografia) ---
print("  Calculando Aspect (Northness & Eastness)...")
aspect_deg = ee.Terrain.aspect(elevation)
aspect_rad = aspect_deg.multiply(3.1415926535 / 180)
aspect_northness = aspect_rad.cos().rename('aspect_northness')
aspect_eastness = aspect_rad.sin().rename('aspect_eastness')

print("Engenharia de features concluída.")


# --- 3. Combinação ---
all_static_predictors = ee.Image.cat([
    aspect_northness,
    aspect_eastness,
])

print("Todas as features estáticas foram combinadas.")

Iniciando engenharia de features estáticas...
  Calculando Aspect (Northness & Eastness)...
Engenharia de features concluída.
Todas as features estáticas foram combinadas.


In [ ]:
# ==============================================================================
# PASSO 6.3 : AMOSTRAGEM E EXPORTAÇÃO 2 (Aspect - com loop anual)(FEITO)
# ==============================================================================

print("Iniciando amostragem e exportação (MODO: 1 TAREFA POR ANO)...")

# A imagem de preditores estáticos (do PASSO 6.2)
all_static_predictors = ee.Image.cat([
    aspect_northness,
    aspect_eastness,
])

output_folder = 'ARTIGO_INCENDIOS_GEE_Aspect' # Pasta de saída no Drive

# --- O Loop de Exportação ---
for i in range(len(list_of_collections)):

    collection_anual = list_of_collections[i]
    ano_string = 2014 + i # Pega o ano (ex: 2014)

    print(f"--- Preparando Tarefa para o Ano: {ano_string} ---")

    # 1. Amostra APENAS a coleção daquele ano
    static_features_dataset_anual = all_static_predictors.sampleRegions(
        collection=collection_anual,
        properties=['DataHora', 'Latitude', 'Longitude'], # Propriedades para manter
        scale=TARGET_RESOLUTION,
        geometries=True # Mantém a geometria para a amostragem
    )

    # 2. Define nomes únicos para a tarefa
    output_filename = f'positive_samples_static_aspect_{ano_string}'
    task_description = f'Export_Static_Aspect_{ano_string}'

    # 3. Cria a tarefa de exportação para o Drive
    task = ee.batch.Export.table.toDrive(
        collection=static_features_dataset_anual,
        description=task_description,
        folder=output_folder,
        fileNamePrefix=output_filename,
        fileFormat='CSV'
    )

    # 4. Inicia a tarefa
    #task.start()
    print(f"Tarefa '{task_description}' ENVIADA.")

print("="*60)
print(f"TODAS AS {len(list_of_collections)} TAREFAS FORAM ENVIADAS!")
print("="*60)
print(">>> INSTRUÇÕES: Vá para a aba 'TASKS' no GEE e clique em 'RUN' em todas elas.")
print("   Você terá 10 CSVs no seu Drive, que pode juntar depois com Pandas.")
print("="*60)

Iniciando amostragem e exportação (MODO: 1 TAREFA POR ANO)...
--- Preparando Tarefa para o Ano: 2014 ---
Tarefa 'Export_Static_Aspect_2014' ENVIADA.
--- Preparando Tarefa para o Ano: 2015 ---
Tarefa 'Export_Static_Aspect_2015' ENVIADA.
--- Preparando Tarefa para o Ano: 2016 ---
Tarefa 'Export_Static_Aspect_2016' ENVIADA.
--- Preparando Tarefa para o Ano: 2017 ---
Tarefa 'Export_Static_Aspect_2017' ENVIADA.
--- Preparando Tarefa para o Ano: 2018 ---
Tarefa 'Export_Static_Aspect_2018' ENVIADA.
--- Preparando Tarefa para o Ano: 2019 ---
Tarefa 'Export_Static_Aspect_2019' ENVIADA.
--- Preparando Tarefa para o Ano: 2020 ---
Tarefa 'Export_Static_Aspect_2020' ENVIADA.
--- Preparando Tarefa para o Ano: 2021 ---
Tarefa 'Export_Static_Aspect_2021' ENVIADA.
--- Preparando Tarefa para o Ano: 2022 ---
Tarefa 'Export_Static_Aspect_2022' ENVIADA.
--- Preparando Tarefa para o Ano: 2023 ---
Tarefa 'Export_Static_Aspect_2023' ENVIADA.
TODAS AS 10 TAREFAS FORAM ENVIADAS!
>>> INSTRUÇÕES: Vá para a aba 'T

In [ ]:
# --------------------------------------------------------------------------
# 3. CRIAR A IMAGEM DE DISTÂNCIA
# --------------------------------------------------------------------------
MAX_DISTANCE = 45000 # 45km

# Esta é a operação pesada
distance_to_roads_image = todas_estradas.distance(
    searchRadius=MAX_DISTANCE
).rename('distance_to_roads')

# Recortar a imagem de distância APENAS para o Cerrado
final_image_to_export = distance_to_roads_image.clip(cerrado_aoi)

print("Imagem de distância de estradas definida.")

# --------------------------------------------------------------------------
# 4. EXPORTAR A IMAGEM COMO UM NOVO ASSET
# --------------------------------------------------------------------------
# Defina o nome e o caminho do SEU NOVO ASSET
asset_id_destino = 'projects/the-byway-476116-n7/assets/distancia_rodovias_rasterV2'
descricao_tarefa = 'Exportar_Raster_Distancia_EstradasV2'

# Aumentar a escala para 90m ajuda a reduzir a chance de erro de memória.
# Se 90m falhar, tente 100m ou 250m.
EXPORT_SCALE_METERS = 90

task = ee.batch.Export.image.toAsset(
    image=final_image_to_export,
    description=descricao_tarefa,
    assetId=asset_id_destino,
    scale=EXPORT_SCALE_METERS,
    region=cerrado_aoi, # Garante que só exporta o Cerrado
    maxPixels=1e13 # Permite exportações muito grandes
)

task.start()

print("============================================================")
print(f"TAREFA ENVIADA! O nome da tarefa é: '{descricao_tarefa}'")
print("============================================================")
print(">>> INSTRUÇÕES:")
print("1. Vá para o GEE Code Editor (https://code.earthengine.google.com).")
print("2. Clique na aba 'Tasks' no painel da direita.")
print(f"3. Clique no botão 'RUN' ao lado da tarefa '{descricao_tarefa}'.")
print("\nAVISO: Esta tarefa pode levar VÁRIAS HORAS para ser concluída.")

Imagem de distância de estradas definida.
TAREFA ENVIADA! O nome da tarefa é: 'Exportar_Raster_Distancia_EstradasV2'
>>> INSTRUÇÕES:
1. Vá para o GEE Code Editor (https://code.earthengine.google.com).
2. Clique na aba 'Tasks' no painel da direita.
3. Clique no botão 'RUN' ao lado da tarefa 'Exportar_Raster_Distancia_EstradasV2'.

AVISO: Esta tarefa pode levar VÁRIAS HORAS para ser concluída.


In [ ]:
# ==============================================================================
# PASSO 6.4 : Juntar com pandas dados do aspect no dataset
# ==============================================================================

import pandas as pd
import glob
import os

aspect_folder_path = '/content/drive/MyDrive/ARTIGO_INCENDIOS_GEE_Aspect'

# Usa 'glob' para encontrar TODOS os CSVs dentro dessa pasta
aspect_csv_files = glob.glob(os.path.join(aspect_folder_path, "*.csv"))

print(f"Encontrados {len(aspect_csv_files)} arquivos CSV de aspect.")

# Cria uma lista para guardar os DataFrames de cada ano
list_of_dfs = []

# Faz um loop, lê cada CSV e adiciona na lista
for f in aspect_csv_files:
    df_anual = pd.read_csv(f)
    list_of_dfs.append(df_anual)

# Concatena todos os DataFrames da lista (um em cima do outro)
df_aspect_total = pd.concat(list_of_dfs, ignore_index=True)

print("Shape do DataFrame de 'aspect' total:", df_aspect_total.shape)
print(df_aspect_total.head())

Encontrados 10 arquivos CSV de aspect.
Shape do DataFrame de 'aspect' total: (604028, 7)
             system:index             DataHora  Latitude  Longitude  \
0  000000000000000001b7_0  2014/01/18 16:39:00   -11.890    -44.804   
1  000000000000000005c0_0  2014/04/03 16:21:00   -12.530    -44.327   
2  000000000000000005c2_0  2014/04/03 16:21:00   -12.810    -44.543   
3  0000000000000000d7bb_0  2014/10/15 16:51:00   -12.806    -44.233   
4  0000000000000000d888_0  2014/10/15 16:51:00   -12.805    -44.222   

   aspect_eastness  aspect_northness  \
0         0.714732     -6.993980e-01   
1        -0.715577      6.985337e-01   
2        -0.898857      4.382424e-01   
3         1.000000      4.489659e-11   
4         0.898853     -4.382494e-01   

                                                .geo  
0  {"geodesic":false,"type":"Point","coordinates"...  
1  {"geodesic":false,"type":"Point","coordinates"...  
2  {"geodesic":false,"type":"Point","coordinates"...  
3  {"geodesic":false,"t

In [ ]:
# ==============================================================================
# PASSO 6.4.1 : Definição do caminho + leitura do .csv principal
# ==============================================================================

# ATUALIZE o caminho para o seu CSV principal
main_csv_path = '/content/drive/MyDrive/Projeto_Incendio/data/raw/positive_samples_aligned_features.csv'

df_main = pd.read_csv(main_csv_path)

print("Shape do DataFrame 'Principal':", df_main.shape)
print(df_main.head())

Shape do DataFrame 'Principal': (604028, 18)
              DataHora  Latitude  Longitude    Municipio Estado    Bioma  \
0  2014/01/18 16:39:00   -11.890    -44.804      ANGICAL  BAHIA  Cerrado   
1  2014/04/03 16:21:00   -12.530    -44.327  BAIAN�POLIS  BAHIA  Cerrado   
2  2014/04/03 16:21:00   -12.810    -44.543  BAIAN�POLIS  BAHIA  Cerrado   
3  2014/10/15 16:51:00   -12.806    -44.233  BAIAN�POLIS  BAHIA  Cerrado   
4  2014/10/15 16:51:00   -12.805    -44.222  BAIAN�POLIS  BAHIA  Cerrado   

   Satelite  elevation_static  slope_static  lulc_class  lst_daily_kelvin  \
0  AQUA_M-T               436      1.325891           4          -9999.00   
1  AQUA_M-T               807      1.327530          21          -9999.00   
2  AQUA_M-T               766      2.115426           4          -9999.00   
3  AQUA_M-T               824      1.901602           4            312.70   
4  AQUA_M-T               828      2.115392           4            313.26   

   ndvi_16day_prefire  temp_2m_C  r

In [ ]:
# ==============================================================================
# PASSO 6.4.2 : Junta ambos(utilizando o join_key, chaves de junção)
# ==============================================================================

# --- A FORMA SEGURA DE FAZER A JUNTA ---

print("Criando chaves de junção (join_key)...")

# 1. Cria a chave no DataFrame Principal
# (Convertemos tudo para string para garantir uma junção exata)
df_main['join_key'] = df_main['DataHora'].astype(str) + \
                      "_" + df_main['Latitude'].astype(str) + \
                      "_" + df_main['Longitude'].astype(str)

# 2. Cria a MESMA chave no DataFrame do Aspect
df_aspect_total['join_key'] = df_aspect_total['DataHora'].astype(str) + \
                              "_" + df_aspect_total['Latitude'].astype(str) + \
                              "_" + df_aspect_total['Longitude'].astype(str)

# 3. Mantenha apenas as colunas que você quer juntar do df_aspect
# (A 'join_key' + as novas features)
df_aspect_para_juntar = df_aspect_total[['join_key', 'aspect_northness', 'aspect_eastness']]

# 4. Faça o MERGE usando a 'join_key'
# how='left' -> Mantém tudo do df_main e anexa o aspect.
df_final = pd.merge(
    df_main,
    df_aspect_para_juntar,
    on='join_key',
    how='left'
)

# 5. Verifique o resultado
print("\nShape do DataFrame 'Final' (após a junta):", df_final.shape)
print("Colunas novas:", ['aspect_northness', 'aspect_eastness'])
print(df_final.head())

# =================================================================
# PASSO FINAL: SALVAR O DATAFRAME DA MEMÓRIA PARA O DRIVE
# =================================================================
print("\nIniciando salvamento do DataFrame final no Google Drive...")

caminho_arquivo_final = '/content/drive/MyDrive/dataset_final_com_aspect.csv'

# index=False é IMPORTANTE para não salvar uma coluna de índice inútil
df_final.to_csv(caminho_arquivo_final, index=False)

print(f"SUCESSO! O arquivo final foi salvo em:")
print(caminho_arquivo_final)



Criando chaves de junção (join_key)...

Shape do DataFrame 'Final' (após a junta): (604028, 21)
Colunas novas: ['aspect_northness', 'aspect_eastness']
              DataHora  Latitude  Longitude    Municipio Estado    Bioma  \
0  2014/01/18 16:39:00   -11.890    -44.804      ANGICAL  BAHIA  Cerrado   
1  2014/04/03 16:21:00   -12.530    -44.327  BAIAN�POLIS  BAHIA  Cerrado   
2  2014/04/03 16:21:00   -12.810    -44.543  BAIAN�POLIS  BAHIA  Cerrado   
3  2014/10/15 16:51:00   -12.806    -44.233  BAIAN�POLIS  BAHIA  Cerrado   
4  2014/10/15 16:51:00   -12.805    -44.222  BAIAN�POLIS  BAHIA  Cerrado   

   Satelite  elevation_static  slope_static  lulc_class  ...  \
0  AQUA_M-T               436      1.325891           4  ...   
1  AQUA_M-T               807      1.327530          21  ...   
2  AQUA_M-T               766      2.115426           4  ...   
3  AQUA_M-T               824      1.901602           4  ...   
4  AQUA_M-T               828      2.115392           4  ...   

   ndvi

In [ ]:
# ==============================================================================
# PASSO 6.4.3 : Salva o dataset com aspect no drive
# ==============================================================================
print("\nIniciando salvamento do DataFrame final no Google Drive...")

caminho_arquivo_final = '/content/drive/MyDrive/Projeto_Incendio/data/rawdataset_final_com_aspect.csv'

# index=False é IMPORTANTE para não salvar uma coluna de índice inútil
df_final.to_csv(caminho_arquivo_final, index=False)

print(f"SUCESSO! O arquivo final foi salvo em:")
print(caminho_arquivo_final)


Iniciando salvamento do DataFrame final no Google Drive...
SUCESSO! O arquivo final foi salvo em:
/content/drive/MyDrive/Projeto_Incendio/data/rawdataset_final_com_aspect.csv


In [ ]:
# --------------------------------------------------------------------------
# 3. CARREGAR O "MAPA" DE PREVISORES DE DISTÂNCIA
# --------------------------------------------------------------------------

# --- 3.1 Carregar os ASSETS PRÉ-CALCULADOS ---
dist_roads_img = ee.Image('projects/the-byway-476116-n7/assets/distancia_rodovias_rasterV2')
dist_water_img = ee.Image('projects/the-byway-476116-n7/assets/distancia_agua_rasteraV2')
dist_urban_img = ee.Image('projects/the-byway-476116-n7/assets/distancia_urban_rasterV2')
print("Assets de distância pré-calculados (Estradas, Água, Urbano) carregados.")

# --- 3.2 Combinar TUDO em uma única imagem de 3 bandas ---
all_distance_predictors = ee.Image.cat([
    dist_roads_img,
    dist_water_img,
    dist_urban_img
])

print("Stack de preditores de distância (3 features) criado com sucesso.")

# --------------------------------------------------------------------------
# 4. EXPORTAÇÃO EM LOOP (A FORMA CORRETA)
# --------------------------------------------------------------------------
TARGET_RESOLUTION = 90
output_folder = 'ARTIGO_INCENDIOS_DISTANCIASV3'

for i in range(len(list_of_collections)):

    collection_anual = list_of_collections[i]
    ano_string = 2014 + i

    print(f"--- Preparando Tarefa de DISTÂNCIA para o Ano: {ano_string} ---")

    # Amostra a coleção do ano contra o "mapa" de 3 features de distância
    distance_features_dataset_anual = all_distance_predictors.sampleRegions(
        collection=collection_anual,
        properties=['DataHora','Latitude', 'Longitude'], # Propriedades para manter
        scale=TARGET_RESOLUTION,
        tileScale=4, # Ajuda a prevenir erros de 'computed value'
        geometries=True # Mantém a geometria para a amostragem
    )

    output_filename = f'positive_samples_static_DISTANCES_{ano_string}_v3'
    task_description = f'Export_Static_DISTANCESv3_{ano_string}'

    task = ee.batch.Export.table.toDrive(
        collection=distance_features_dataset_anual,
        description=task_description,
        folder=output_folder,
        fileNamePrefix=output_filename,
        fileFormat='CSV'
    )

    task.start()
    print(f"Tarefa '{task_description}' ENVIADA.")

print("="*60)
print(f"TODAS AS {len(list_of_collections)} TAREFAS DE DISTÂNCIA FORAM ENVIADAS!")
print("="*60)
print(">>> INSTRUÇÕES: Vá para a aba 'TASKS' no GEE e clique 'RUN' em todas.")

Assets de distância pré-calculados (Estradas, Água, Urbano) carregados.
Stack de preditores de distância (3 features) criado com sucesso.
--- Preparando Tarefa de DISTÂNCIA para o Ano: 2014 ---
Tarefa 'Export_Static_DISTANCESv3_2014' ENVIADA.
--- Preparando Tarefa de DISTÂNCIA para o Ano: 2015 ---
Tarefa 'Export_Static_DISTANCESv3_2015' ENVIADA.
--- Preparando Tarefa de DISTÂNCIA para o Ano: 2016 ---
Tarefa 'Export_Static_DISTANCESv3_2016' ENVIADA.
--- Preparando Tarefa de DISTÂNCIA para o Ano: 2017 ---
Tarefa 'Export_Static_DISTANCESv3_2017' ENVIADA.
--- Preparando Tarefa de DISTÂNCIA para o Ano: 2018 ---
Tarefa 'Export_Static_DISTANCESv3_2018' ENVIADA.
--- Preparando Tarefa de DISTÂNCIA para o Ano: 2019 ---
Tarefa 'Export_Static_DISTANCESv3_2019' ENVIADA.
--- Preparando Tarefa de DISTÂNCIA para o Ano: 2020 ---
Tarefa 'Export_Static_DISTANCESv3_2020' ENVIADA.
--- Preparando Tarefa de DISTÂNCIA para o Ano: 2021 ---
Tarefa 'Export_Static_DISTANCESv3_2021' ENVIADA.
--- Preparando Tarefa 

In [3]:
import pandas as pd

df = pd.read_csv("/content/drive/MyDrive/Projeto_Incendio/data/raw/dataset_focos_com_distanciasFORREAL.csv")

lat_min = df["Latitude"].min()
lat_max = df["Latitude"].max()
lon_min = df["Longitude"].min()
lon_max = df["Longitude"].max()

print(lat_min, lat_max, lon_min, lon_max)


-24.67258 -2.40147 -60.096 -41.54794


In [4]:

REGION = ee.Geometry.Rectangle([lon_min, lat_min, lon_max, lat_max])
print("REGION definida.")

SCALE_5KM = 5000  # ~5 km

# Imagem com latitude/longitude por pixel
lonlat = ee.Image.pixelLonLat()

# Amostra de pontos (um por “pixel” de 5 km) dentro da REGION
grid_5km = lonlat.sample(
    region=REGION,
    scale=SCALE_5KM,
    geometries=True
)

print("Tamanho aproximado do grid 5 km:", grid_5km.size().getInfo())


REGION definida.
Tamanho aproximado do grid 5 km: 206228


In [7]:
# --- 3.3 DADOS TOPOGRÁFICOS (SRTM) ---
# Elevação é uma imagem única (estática), não uma coleção.
elevation = ee.Image('USGS/SRTMGL1_003').select('elevation')
# Cálculo do declive (slope) diretamente a partir da elevação.
slope = ee.Terrain.slope(elevation)

print("Dados de elevação e declive carregados.")


# 1. Renomeia as imagens estáticas ANTES da função
elevation_renamed = elevation.rename('elevation_static')
slope_renamed = slope.rename('slope_static')

dist_roads   = ee.Image("projects/the-byway-476116-n7/assets/distancia_rodovias_rasterV3").rename("distance_to_roads")
dist_urban   = ee.Image("projects/the-byway-476116-n7/assets/distancia_urban_rasterV3").rename("distance_to_urban")
dist_water   = ee.Image("projects/the-byway-476116-n7/assets/distancia_agua_rasteraV3").rename("distance_to_water")
# Já usando o que você calculou
aspect_deg = ee.Terrain.aspect(elevation)
aspect_rad = aspect_deg.multiply(3.1415926535 / 180)
aspect_northness = aspect_rad.cos().rename('aspect_northness')
aspect_eastness  = aspect_rad.sin().rename('aspect_eastness')

static_predictors = ee.Image.cat([
    elevation_renamed,
    slope_renamed,
    aspect_northness,
    aspect_eastness,
    dist_roads,
    dist_urban,
    dist_water,
])

def export_static_5km():
    def sample_static(feat):
        geom = feat.geometry()
        sampled = static_predictors.sample(
            region=geom,
            scale=30,
            numPixels=1
        ).first()
        return feat.setMulti(sampled)

    grid_with_static = grid_5km.map(sample_static)

    task = ee.batch.Export.table.toDrive(
        collection=grid_with_static,
        description="static_5km_features",
        folder="Projeto_Incendio",
        fileNamePrefix="static_5km_features",
        fileFormat="CSV"
    )
    task.start()
    print("Tarefa de export estático iniciada.")


Dados de elevação e declive carregados.


In [8]:
export_static_5km()

Tarefa de export estático iniciada.


In [ ]:
def make_daily_features_5km(date):
    """Gera features em 5 km para UM dia."""
    date = ee.Date(date)

    # String no formato esperado pela align_all_features
    datahora_str = date.format('YYYY/MM/dd 15:00:00')  # horário fixo, só pra alinhar ERA5/LST

    # Coloca a propriedade DataHora em cada célula do grid
    def set_datetime(feat):
        return feat.set('DataHora', datahora_str)

    grid_with_datetime = grid_5km.map(set_datetime)

    # Aplica a MESMA função dos focos
    enriched = grid_with_datetime.map(align_all_features)

    return enriched


In [ ]:
def export_features_5km_for_year(year):
    """
    Gera e exporta features em 5 km para TODOS os dias de um ano.
    Cria um CSV no Drive: features_5km_<year>.csv (na pasta Projeto_Incendio).
    """
    year = int(year)
    print(f"Preparando ano {year}...")

    start = ee.Date.fromYMD(year, 1, 1)
    end   = start.advance(1, 'year')  # exclusive

    # Lista de dias do ano
    n_days = end.difference(start, 'day')
    dates = ee.List.sequence(0, n_days.subtract(1)).map(
        lambda d: start.advance(d, 'day')
    )

    # Para cada dia, gera uma FC com o grid enriquecido
    daily_collections = dates.map(make_daily_features_5km)

    # Achata tudo num único FeatureCollection
    fc_year = ee.FeatureCollection(daily_collections).flatten()

    print("Número estimado de features no ano:", fc_year.size().getInfo())

    task = ee.batch.Export.table.toDrive(
        collection=fc_year,
        description=f"features_5km_{year}",
        folder="Projeto_Incendio",
        fileNamePrefix=f"features_5km_{year}",
        fileFormat="CSV"
    )

    task.start()
    print(f"Tarefa de exportação iniciada para o ano {year}.")
    return task


In [ ]:
#tasks = []
#for year in range(2014, 2024):  # 2014 a 2023
    #tasks.append(export_features_5km_for_year(year))


In [2]:
#Exportar mensalmente
def export_features_5km_for_year_monthly(year):
    year = int(year)
    print(f"Preparando exports mensais para {year}...")

    tasks = []

    for month in range(1, 13):
        start = ee.Date.fromYMD(year, month, 1)
        end   = start.advance(1, 'month')  # exclusive

        # lista de dias do mês
        n_days = end.difference(start, 'day')
        dates = ee.List.sequence(0, n_days.subtract(1)).map(
            lambda d: start.advance(d, 'day')
        )

        # gera features por dia nesse mês
        daily_collections = dates.map(make_daily_features_5km)
        fc_month = ee.FeatureCollection(daily_collections).flatten()

        print(f"Ano {year}, mês {month:02d} -> size aproximado:", fc_month.size().getInfo())

        task = ee.batch.Export.table.toDrive(
            collection=fc_month,
            description=f"features_5km_{year}_{month:02d}",
            folder="Projeto_Incendio",
            fileNamePrefix=f"features_5km_{year}_{month:02d}",
            fileFormat="CSV"
        )
        task.start()
        tasks.append(task)
        print(f"Tarefa iniciada: features_5km_{year}_{month:02d}")

    return tasks


In [ ]:
tasks_2015 = export_features_5km_for_year_monthly(2015)
tasks_2016 = export_features_5km_for_year_monthly(2016)


In [ ]:
import pandas as pd

# --- 1. Configuração ---
ARQUIVO_1_PATH = '/content/drive/MyDrive/ARTIGO_INCENDIOS_DISTANCIASV3 (1)/positive_samples_static_DISTANCES_2014_v3.csv'
ARQUIVO_2_PATH = '/content/drive/MyDrive/Projeto_Incendio/data/raw/rawdataset_final_com_aspect.csv'

COLUNAS_ARQUIVO_1 = {'Latitude': 'Latitude', 'Longitude': 'Longitude'}
COLUNAS_ARQUIVO_2 = {'Latitude': 'Latitude', 'Longitude': 'Longitude'}

CASAS_DECIMAIS = 5

# --- 2. Carregar e Preparar os Dados ---
try:
    df1 = pd.read_csv(ARQUIVO_1_PATH)
    df2 = pd.read_csv(ARQUIVO_2_PATH)
except FileNotFoundError as e:
    print(f"Erro: Arquivo não encontrado. Verifique o caminho: {e.filename}")
    exit()

# Renomeia colunas para um padrão comum (ex: 'lat' e 'lon')
print(df1.columns)

# --- 3. Arredondar e Comparar ---

# Arredonda as coordenadas para evitar problemas de precisão de float
# (Ex: 45.123456 vs 45.123457)
df1['lat_rounded'] = df1['Latitude'].round(CASAS_DECIMAIS)
df1['lon_rounded'] = df1['Longitude'].round(CASAS_DECIMAIS)
df2['lat_rounded'] = df2['Latitude'].round(CASAS_DECIMAIS)
df2['lon_rounded'] = df2['Longitude'].round(CASAS_DECIMAIS)

# Chaves de comparação
chaves = ['lat_rounded', 'lon_rounded']

# 4. Fazer o "Merge" para encontrar diferenças
# how='outer' -> Pega TUDO de ambos os dataframes
# indicator=True -> Cria uma coluna '_merge' que diz a origem
comparacao = pd.merge(
    df1,
    df2,
    on=chaves,
    how='outer',
    indicator=True
)

# --- 5. Analisar os Resultados ---

print("Análise da Comparação:")

# Filtra para ver o que existe em AMBOS
matching = comparacao[comparacao['_merge'] == 'both']
print(f"\nCoordenadas que BATEM (em ambos): {len(matching)} linhas")

# Filtra para ver o que SÓ existe no Arquivo 1
so_no_1 = comparacao[comparacao['_merge'] == 'left_only']
print(f"Coordenadas SÓ no Arquivo 1: {len(so_no_1)} linhas")
if len(so_no_1) > 0:
    print(so_no_1[chaves].head()) # Mostra as 5 primeiras

# Filtra para ver o que SÓ existe no Arquivo 2
so_no_2 = comparacao[comparacao['_merge'] == 'right_only']
print(f"\nCoordenadas SÓ no Arquivo 2: {len(so_no_2)} linhas")
if len(so_no_2) > 0:
    print(so_no_2[chaves].head()) # Mostra as 5 primeiras

diferencas = comparacao[comparacao['_merge'] != 'both']
diferencas.to_csv('diferencas_de_coordenadas.csv', index=False)
print(f"\nArquivo 'diferencas_de_coordenadas.csv' salvo com {len(diferencas)} linhas.")

Index(['DataHora', 'Latitude', 'Longitude', 'Municipio', 'Estado', 'Bioma',
       'Satelite', 'elevation_static', 'slope_static', 'lulc_class',
       'lst_daily_kelvin', 'ndvi_16day_prefire', 'temp_2m_C',
       'relative_humidity_percent', 'wind_u_ms', 'wind_v_ms', 'wind_speed_ms',
       'precipitation_hourly_mm'],
      dtype='object')
Análise da Comparação:

Coordenadas que BATEM (em ambos): 608434 linhas
Coordenadas SÓ no Arquivo 1: 0 linhas

Coordenadas SÓ no Arquivo 2: 0 linhas

Arquivo 'diferencas_de_coordenadas.csv' salvo com 0 linhas.


In [ ]:
import numpy as np
import pandas as pd

df = pd.read_csv("/content/drive/MyDrive/Projeto_Incendio/data/raw/dataset_focos_com_distanciasFORREAL.csv")

cols_nodata = [
    "elevation_static",
    "slope_static",
    "lst_daily_kelvin",
    "ndvi_16day_prefire",
    "temp_2m_C",
    "relative_humidity_percent",
    "wind_u_ms",
    "wind_v_ms",
    "wind_speed_ms",
    "precipitation_hourly_mm",
    "aspect_northness",
    "aspect_eastness"
]

df[cols_nodata] = df[cols_nodata].replace(-9999, np.nan)


In [ ]:
missing_rate = df[cols_nodata].isna().mean().sort_values(ascending=False)
print(missing_rate)


lst_daily_kelvin             0.285137
wind_v_ms                    0.000096
wind_u_ms                    0.000096
temp_2m_C                    0.000096
precipitation_hourly_mm      0.000096
ndvi_16day_prefire           0.000007
relative_humidity_percent    0.000005
slope_static                 0.000000
elevation_static             0.000000
wind_speed_ms                0.000000
aspect_northness             0.000000
aspect_eastness              0.000000
dtype: float64


In [ ]:
df["lst_missing_flag"] = df["lst_daily_kelvin"].isna().astype(int)


In [ ]:
df["lst_daily_kelvin"] = df["lst_daily_kelvin"].replace(-9999, pd.NA)

# extrair o mês
df["month"] = pd.to_datetime(df["DataHora"]).dt.month

# calcular mediana de LST por mês (no Cerrado inteiro)
group_median = df.groupby("month")["lst_daily_kelvin"].transform("median")

# 1º: preencher NaN com a mediana daquele mês
df["lst_daily_kelvin"] = df["lst_daily_kelvin"].fillna(group_median)

# 2º: se sobrar algum NaN (mês raro etc.), usa mediana global
df["lst_daily_kelvin"] = df["lst_daily_kelvin"].fillna(df["lst_daily_kelvin"].median())

In [ ]:
df.to_csv("/content/drive/MyDrive/Projeto_Incendio/data/raw/positive_samples_clean.csv", index=False)

In [ ]:
import pandas as pd
import numpy as np

df = pd.read_csv("/content/drive/MyDrive/Projeto_Incendio/data/raw/positive_samples_clean.csv")
print(df.shape)      # (n_linhas, n_colunas)
print(df.dtypes)     # tipos de cada coluna


(604028, 26)
DataHora                      object
Latitude                     float64
Longitude                    float64
Municipio                     object
Estado                        object
Bioma                         object
Satelite                      object
elevation_static               int64
slope_static                 float64
lulc_class                     int64
lst_daily_kelvin             float64
ndvi_16day_prefire           float64
temp_2m_C                    float64
relative_humidity_percent    float64
wind_u_ms                    float64
wind_v_ms                    float64
wind_speed_ms                float64
precipitation_hourly_mm      float64
join_key                      object
aspect_northness             float64
aspect_eastness              float64
distance_to_roads            float64
distance_to_urban            float64
distance_to_water            float64
lst_missing_flag               int64
month                          int64
dtype: object


In [ ]:
cols_check = [
    "elevation_static",
    "slope_static",
    "lst_daily_kelvin",
    "ndvi_16day_prefire",
    "temp_2m_C",
    "relative_humidity_percent",
    "wind_u_ms",
    "wind_v_ms",
    "wind_speed_ms",
    "precipitation_hourly_mm",
    "aspect_northness",
    "aspect_eastness",
    "distance_to_roads",
    "distance_to_urban",
    "distance_to_water",
]

for col in cols_check:
    n_neg9999 = (df[col] == -9999).sum()
    print(col, "tem", n_neg9999, "valores -9999")


elevation_static tem 0 valores -9999
slope_static tem 0 valores -9999
lst_daily_kelvin tem 0 valores -9999
ndvi_16day_prefire tem 0 valores -9999
temp_2m_C tem 0 valores -9999
relative_humidity_percent tem 0 valores -9999
wind_u_ms tem 0 valores -9999
wind_v_ms tem 0 valores -9999
wind_speed_ms tem 0 valores -9999
precipitation_hourly_mm tem 0 valores -9999
aspect_northness tem 0 valores -9999
aspect_eastness tem 0 valores -9999
distance_to_roads tem 0 valores -9999
distance_to_urban tem 0 valores -9999
distance_to_water tem 0 valores -9999


In [ ]:
missing_rate = df.isna().mean().sort_values(ascending=False)
print(missing_rate)


distance_to_urban            0.207048
distance_to_water            0.033917
distance_to_roads            0.000639
temp_2m_C                    0.000096
precipitation_hourly_mm      0.000096
wind_v_ms                    0.000096
wind_u_ms                    0.000096
ndvi_16day_prefire           0.000007
relative_humidity_percent    0.000005
DataHora                     0.000000
lulc_class                   0.000000
slope_static                 0.000000
elevation_static             0.000000
Satelite                     0.000000
Bioma                        0.000000
Estado                       0.000000
Municipio                    0.000000
Longitude                    0.000000
Latitude                     0.000000
lst_daily_kelvin             0.000000
wind_speed_ms                0.000000
join_key                     0.000000
aspect_eastness              0.000000
aspect_northness             0.000000
lst_missing_flag             0.000000
month                        0.000000
dtype: float

In [ ]:
df = df.dropna(subset=["distance_to_roads", "distance_to_urban", "distance_to_water"])
df.to_csv("/content/drive/MyDrive/Projeto_Incendio/data/raw/positive_samples_clean_FINAL.csv", index=False)

In [ ]:
missing_rate = df.isna().mean().sort_values(ascending=False)
print(missing_rate)

precipitation_hourly_mm      0.000125
temp_2m_C                    0.000125
wind_u_ms                    0.000125
wind_v_ms                    0.000125
ndvi_16day_prefire           0.000009
relative_humidity_percent    0.000006
Longitude                    0.000000
Municipio                    0.000000
DataHora                     0.000000
Latitude                     0.000000
lulc_class                   0.000000
slope_static                 0.000000
elevation_static             0.000000
Satelite                     0.000000
Bioma                        0.000000
Estado                       0.000000
lst_daily_kelvin             0.000000
wind_speed_ms                0.000000
join_key                     0.000000
aspect_northness             0.000000
aspect_eastness              0.000000
distance_to_roads            0.000000
distance_to_urban            0.000000
distance_to_water            0.000000
lst_missing_flag             0.000000
month                        0.000000
dtype: float

In [ ]:
print("month min/max:", df["month"].min(), df["month"].max())
print(df["lst_missing_flag"].value_counts(dropna=False))


month min/max: 1 12
lst_missing_flag
0    329465
1    135470
Name: count, dtype: int64


In [ ]:
# ver alguns casos onde flag = 1
print(df.loc[df["lst_missing_flag"] == 1, ["DataHora", "lst_daily_kelvin"]].head())


              DataHora  lst_daily_kelvin
0  2014/01/18 16:39:00            306.48
1  2014/04/03 16:21:00            305.00
2  2014/04/03 16:21:00            305.00
5  2014/11/11 16:33:00            310.44
6  2014/11/11 16:33:00            310.44


In [ ]:
print("NDVI:", df["ndvi_16day_prefire"].min(), df["ndvi_16day_prefire"].max())
print("Temp 2m:", df["temp_2m_C"].min(), df["temp_2m_C"].max())
print("RH%:", df["relative_humidity_percent"].min(), df["relative_humidity_percent"].max())
print("LST:", df["lst_daily_kelvin"].min(), df["lst_daily_kelvin"].max())
print("Dist roads:", df["distance_to_roads"].min(), df["distance_to_roads"].max())
print("Dist urban:", df["distance_to_urban"].min(), df["distance_to_urban"].max())
print("Dist water:", df["distance_to_water"].min(), df["distance_to_water"].max())


NDVI: -0.0324 0.945
Temp 2m: 8.23816833496096 42.13675842285159
RH%: 7.341215781167571 100.0
LST: 282.68 351.34000000000003
Dist roads: 0.032747641449211 40975.30128009974
Dist urban: 0.0 31819.805153394635
Dist water: 0.0 31756.22931016841


In [ ]:

# 1) Carrega o CSV já com LST imputado e flags criadas
df = pd.read_csv("/content/drive/MyDrive/Projeto_Incendio/data/raw/positive_samples_clean_FINAL.csv")
# 2) Colunas que ainda tinham NaN
cols_drop = [
    "precipitation_hourly_mm",
    "temp_2m_C",
    "wind_u_ms",
    "wind_v_ms",
    "ndvi_16day_prefire",
    "relative_humidity_percent"
]

print("Antes de dropar:", df.shape)

# 3) Dropar apenas as linhas com falta nessas colunas
df_clean = df.dropna(subset=cols_drop).reset_index(drop=True)

print("Depois de dropar:", df_clean.shape)

# 4) conferir se ainda sobrou algum NaN em qualquer coluna
print(df_clean.isna().mean().sort_values(ascending=False).head(10))

# 5) Salvar versão limpinha
df_clean.to_csv("/content/drive/MyDrive/Projeto_Incendio/data/raw/positive_samples_clean_ForREAL.csv", index=False)


Antes de dropar: (464935, 26)
Depois de dropar: (464870, 26)
DataHora            0.0
Latitude            0.0
Longitude           0.0
Municipio           0.0
Estado              0.0
Bioma               0.0
Satelite            0.0
elevation_static    0.0
slope_static        0.0
lulc_class          0.0
dtype: float64
